In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd

from pymc_marketing.mmm import MMM, GeometricAdstock, LogisticSaturation
from mmm_eval.data.synth_data_generator import generate_data
from mmm_eval.configs import PyMCConfig
from pymc_marketing.prior import Prior
from mmm_eval import run_evaluation

In [2]:
data = generate_data()

X = data.drop(columns=["revenue","quantity"])
y = data["quantity"]

In [3]:
base_model = MMM(
    date_column="date_week" ,
    channel_columns=["channel_1","channel_2"],
    adstock=GeometricAdstock(l_max=4),
    saturation=LogisticSaturation()
)

# base_model.fit(X=X, y=y, chains=4, target_accept=0.85)



In [4]:
# We dont need X and y - we'll get those from the input data!
fit_kwargs = { 
    "chains": 4,
    "target_accept": 0.85,
}

base_config = PyMCConfig.from_model_object(base_model, fit_kwargs=fit_kwargs, response_column="quantity", revenue_column="revenue")

In [ ]:
results = run_evaluation("pymc-marketing", data, base_config) 